In [1]:
import dask.bag as bag

encoding = "cp1252"
raw_data = bag.read_text("finefoods.txt.gz", encoding=encoding)
raw_data

dask.bag<bag-from-delayed, npartitions=1>

In [2]:
raw_data.take(10)

('product/productId: B001E4KFG0\n',
 'review/userId: A3SGXH7AUHU8GW\n',
 'review/profileName: delmartian\n',
 'review/helpfulness: 1/1\n',
 'review/score: 5.0\n',
 'review/time: 1303862400\n',
 'review/summary: Good Quality Dog Food\n',
 'review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.\n',
 '\n',
 'product/productId: B00813GRG4\n')

In [3]:
raw_data.count().compute()

5116093

In [4]:
from dask.delayed import delayed

def get_next_part(file, start_index, encoding, span_index=0, blocksize=4096):
    file.seek(start_index)
    buffer = file.read(blocksize+span_index).decode(encoding)
    delimiter_position = buffer.find("\n\n")

    if delimiter_position == -1:
        return get_next_part(file, start_index, encoding, span_index + blocksize)
    else:
        file.seek(start_index)
        return start_index, delimiter_position

In [ ]:
import gzip

with gzip.open("finefoods.txt.gz", "rb") as file_handle:
    size = file_handle.seek(0,2) - 1
    print(size)
    more_data = True
    output = []
    current_position = next_position = 0
    while more_data:
        if current_position >= size:
            more_data = False
        else:
            current_position, next_position = get_next_part(file_handle, current_position, encoding, 0)
            output.append((current_position, next_position))
            current_position = current_position + next_position + 2

370796483
